## Dask

Dask é uma biblioteca em Python para computação paralela e distribuída, projetada para escalar facilmente de um notebook local até clusters de grande porte. Ele permite processar grandes volumes de dados que não cabem na memória RAM, utilizando uma interface familiar ao ecossistema do Python (como pandas, NumPy e scikit-learn).

Principais características do Dask:
✅ Compatível com Pandas e NumPy: Dask oferece dask.dataframe e dask.array, que imitam as APIs de pandas e NumPy, mas funcionam de forma preguiçosa (lazy), processando os dados em partes (chamadas de partições).

🚀 Execução paralela: Usa múltiplos núcleos da CPU para acelerar tarefas pesadas.

🧠 Execução distribuída: Pode ser usado com clusters (como Kubernetes, Yarn, ou com Dask.distributed) para rodar código em várias máquinas.

📦 Integração com machine learning: Funciona bem com scikit-learn e outras bibliotecas para treinar modelos em conjuntos de dados muito grandes.

### Cluster Local
Um cluster local do Dask é um ambiente de execução paralelo criado na sua própria máquina. Ele simula um "mini cluster", mas tudo roda localmente — é uma forma prática de aproveitar todos os núcleos da sua CPU para acelerar tarefas.


In [1]:
from dask.distributed import LocalCluster, Client
from dask.diagnostics import ProgressBar
import dask.dataframe as dd

In [2]:
path = 'C:\\Users\\mateu\\Documents\\MEGA\\Projetos\\HELPS\\MiniBook\\DataAnalise\\dataset\\dados.csv'

## 🔧 `LocalCluster` – Parâmetros principais

```python
from dask.distributed import LocalCluster

cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit='2GB',
    processes=True,
    scheduler_port=0,
    dashboard_address=':8787',
    silence_logs='error',
    local_directory=None,
    name="meu_cluster",
)
```


### 📌 Explicando cada parâmetro:

| Parâmetro               | Tipo        | Descrição |
|-------------------------|-------------|-----------|
| **`n_workers`**         | `int`       | Número de *workers* (processos ou threads independentes). Cada worker executa tarefas em paralelo. |
| **`threads_per_worker`** | `int`      | Número de **threads por worker**. Se for 1, cada worker executa uma tarefa por vez. |
| **`memory_limit`**      | `str` ou `float` | Limite de memória por worker. Pode ser em GB (`'2GB'`), MB (`'512MB'`) ou um número em bytes. |
| **`processes`**         | `bool`      | Define se os workers devem ser **processos** separados (`True`, padrão) ou **threads** (`False`). |
| **`scheduler_port`**    | `int`       | Porta do scheduler. Se `0`, ele escolhe uma automaticamente. |
| **`dashboard_address`** | `str`       | Endereço do dashboard. Ex: `':8787'` (localhost:8787), ou `'127.0.0.1:0'` (porta aleatória). |
| **`silence_logs`**      | `str`       | Nível de log para silenciar mensagens. Pode ser `'error'`, `'warning'`, `'info'`, etc. |
| **`local_directory`**   | `str`       | Caminho local para armazenar arquivos temporários dos workers (como cache). |
| **`name`**              | `str`       | Nome do cluster (aparece no dashboard). |


Criando o cluster local

In [ ]:
cluster = LocalCluster(
    n_workers         =6,
    threads_per_worker=2,
    memory_limit='1GB',
    name        ="Cluster Local",
    local_directory="/tmp/dask",
)

client = Client(cluster)

2025-04-05 18:56:38,169 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 94cbf0893a280c223827fca05349b191 initialized by task ('shuffle-transfer-94cbf0893a280c223827fca05349b191', 0) executed on worker tcp://127.0.0.1:62277
2025-04-05 18:56:44,025 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62271 (pid=7348) exceeded 95% memory budget. Restarting...
2025-04-05 18:56:44,090 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:62271' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('p2pshuffle-94cbf0893a280c223827fca05349b191', 4)} (stimulus_id='handle-worker-cleanup-1743890204.0899389')
2025-04-05 18:56:44,093 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 94cbf0893a280c223827fca05349b191 deactivated due to stimulus 'handle-worker-cleanup-1743890204.0899389'
2025-04-05 18:56:44,097 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 94cbf0893a280c223827fca05349b191 restarted due to

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 12,Total memory: 5.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62195,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:62269,Total threads: 2
Dashboard: http://127.0.0.1:62278/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:62199,


In [5]:
data = dd.read_csv(path, delimiter=',')

`npartitions` : é definido pelo `n_workers` + `threads_per_worker`

In [6]:
data = data.repartition(npartitions=12)

In [7]:
data.head(2)

,id,ano,nome,preço,prime,empresa,desconto,estado
0,9001763654,1973,Evelyn Fogaça,3587,False,nissan,0.05,Alagoas
1,8096557597,2002,Sr. Davi Lucca Rezende,1289,False,nissin,0.00,Pernambuco


In [8]:
with ProgressBar():
    descricao = data.describe().compute()

descricao

,id,ano,preço,desconto
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
mean,4.999143e+09,1.997131e+03,2.999507e+03,4.999308e-02
std,2.887131e+09,1.595329e+01,1.155006e+03,4.081858e-02
min,2.616000e+03,1.970000e+03,1.000000e+03,0.000000e+00
25%,2.510162e+09,1.983000e+03,2.002000e+03,0.000000e+00
50%,5.006660e+09,1.997000e+03,3.004000e+03,5.000000e-02
75%,7.507617e+09,2.011000e+03,4.004000e+03,1.000000e-01
max,1.000000e+10,2.025000e+03,4.999000e+03,1.000000e-01


In [9]:
data.groupby(by=['empresa'])['preço'].mean().compute()

empresa
brastemp    2998.400483
nissan      2999.779444
nissin      2999.596394
toyota      3000.252135
Name: preço, dtype: float64

In [10]:
def pow(x, lamb=2):
    return x**lamb

data['preço'].apply(lambda x: x, meta=('preço','int64')).compute()

0          3587
1          1289
2          1343
3          4538
4          2710
           ... 
1000074    3990
1000075    4650
1000076    3471
1000077    1842
1000078    1034
Name: preço, Length: 10000000, dtype: int64